# Simulation & Analysis: Adsorption Modeling for Water Treatment

**Author:** Cayleigh Sitchon

**Project:** Simulating adsorption kinetics in a batch reactor using Langmuir and Freundlich isotherms. This notebook is a full research workflow: model definition, simulation, synthetic dataset generation, visualization, isotherm fitting, and conclusions.

**Notes:** This notebook is self-contained and includes function definitions; you can also import `isotherm_functions.py` from the repository if you prefer to keep code modular.


## Problem statement

We model adsorption in a well-mixed batch reactor to understand how a sorbent removes dissolved contaminants from water. Specifically, we: 

- Implement Langmuir and Freundlich isotherm models for equilibrium adsorption. 
- Simulate kinetically-limited adsorption with an ODE system for concentration in solution (C) and adsorbed amount (q).
- Generate a synthetic dataset by sampling realistic parameter ranges and simulating many virtual experiments.
- Fit isotherm models to equilibrium data and visualize results.


## Isotherm models

**Langmuir isotherm:**

\[ q^*(C) = \frac{q_{max} b C}{1 + b C} \]

**Freundlich isotherm:**

\[ q^*(C) = K_f C^{1/n} \]

where \(C\) is aqueous concentration (mg/L) and \(q\) is adsorbed amount (mg/g).


In [ ]:
# Imports & helper functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
import seaborn as sns

sns.set(style='whitegrid', rc={'figure.figsize':(8,5)})

# -----------------------------
# Isotherm functions (same as isotherm_functions.py)
# -----------------------------
def langmuir_isotherm(C, q_max, b):
    Cpos = np.maximum(C, 0.0)
    return (q_max * b * Cpos) / (1.0 + b * Cpos)

def freundlich_isotherm(C, Kf, n):
    Cpos = np.maximum(C, 0.0)
    return Kf * (Cpos ** (1.0 / n))

# -----------------------------
# ODE system (kinetic batch model)
# -----------------------------
def adsorption_odes(t, y, params, isotherm='langmuir'):
    C, q = y
    V = params['V']
    m = params['m']
    k_ads = params['k_ads']
    
    if isotherm == 'langmuir':
        q_star = langmuir_isotherm(C, params['q_max'], params['b'])
    elif isotherm == 'freundlich':
        q_star = freundlich_isotherm(C, params['Kf'], params['n'])
    else:
        raise ValueError("isotherm must be 'langmuir' or 'freundlich'")
    
    dqdt = k_ads * (q_star - q)
    dCdt = - (m / V) * dqdt
    return [dCdt, dqdt]

def run_simulation(params, C0=50.0, q0=0.0, t_span=(0,200), t_eval=None, isotherm='langmuir', solver='BDF'):
    if t_eval is None:
        t_eval = np.linspace(t_span[0], t_span[1], 500)
    y0 = [C0, q0]
    sol = solve_ivp(adsorption_odes, t_span, y0, args=(params, isotherm),
                    t_eval=t_eval, method=solver, rtol=1e-6, atol=1e-9)
    if not sol.success:
        raise RuntimeError("ODE solver failed: " + str(sol.message))
    return sol


In [ ]:
# Example simulation with chosen parameters (visualize time-series)
params_example = {
    'V': 1.0,         # L
    'm': 1.0,         # g
    'k_ads': 0.1,     # 1/min
    'q_max': 200.0,   # mg/g (Langmuir)
    'b': 0.05,        # L/mg (Langmuir)
    'Kf': 30.0,       # Freundlich Kf
    'n': 2.0          # Freundlich n
}

C0_example = 50.0
t_span = (0, 200)
t_eval = np.linspace(t_span[0], t_span[1], 500)

solL = run_simulation(params_example, C0=C0_example, t_span=t_span, t_eval=t_eval, isotherm='langmuir')
solF = run_simulation(params_example, C0=C0_example, t_span=t_span, t_eval=t_eval, isotherm='freundlich')

# Plot Langmuir result
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(solL.t, solL.y[0], label='C (mg/L)')
plt.plot(solL.t, solL.y[1], label='q (mg/g)')
plt.xlabel('Time (min)'); plt.title('Langmuir (example)'); plt.legend()

# Plot Freundlich result
plt.subplot(1,2,2)
plt.plot(solF.t, solF.y[0], label='C (mg/L)')
plt.plot(solF.t, solF.y[1], label='q (mg/g)')
plt.xlabel('Time (min)'); plt.title('Freundlich (example)'); plt.legend()
plt.tight_layout()
plt.show()

print(f"Langmuir final C = {solL.y[0,-1]:.3f} mg/L, q = {solL.y[1,-1]:.3f} mg/g")

In [ ]:
# Synthetic dataset generation function
def generate_synthetic_dataset(num_samples=300, isotherm='langmuir', t_span=(0,200), t_eval=None, random_seed=42):
    np.random.seed(random_seed)
    results = []
    if t_eval is None:
        t_eval = np.linspace(t_span[0], t_span[1], 500)
    
    for _ in range(num_samples):
        params = {
            'V': 1.0,
            'm': np.random.uniform(0.5, 5.0),            # adsorbent mass (g)
            'k_ads': np.random.uniform(0.01, 0.2),       # 1/min
            'q_max': 200.0,
            'b': np.random.uniform(0.01, 0.12),          # Langmuir b (L/mg)
            'Kf': np.random.uniform(10.0, 50.0),         # Freundlich Kf
            'n': np.random.uniform(1.0, 3.0)             # Freundlich n
        }
        C0 = np.random.uniform(10.0, 100.0)
        sol = run_simulation(params, C0=C0, q0=0.0, t_span=t_span, t_eval=t_eval, isotherm=isotherm)
        if not sol.success:
            continue
        C_final = float(sol.y[0, -1])
        q_final = float(sol.y[1, -1])
        removal_pct = 100.0 * (C0 - C_final) / C0
        results.append({
            'C0': C0, 'm': params['m'], 'k_ads': params['k_ads'],
            'q_final': q_final, 'C_final': C_final, 'removal_pct': removal_pct,
            'isotherm': isotherm, 'q_max': params['q_max'], 'b': params['b'],
            'Kf': params['Kf'], 'n': params['n']
        })
    return pd.DataFrame(results)

# Generate datasets
df_lang = generate_synthetic_dataset(num_samples=300, isotherm='langmuir')
df_fre = generate_synthetic_dataset(num_samples=300, isotherm='freundlich')

# Save CSVs for convenience
df_lang.to_csv('synthetic_dataset_langmuir.csv', index=False)
df_fre.to_csv('synthetic_dataset_freundlich.csv', index=False)

print('Generated datasets: ', df_lang.shape, df_fre.shape)

In [ ]:
# Exploratory Data Analysis (EDA)
df = df_lang.copy()  # use Langmuir synthetic data as example

# Scatter: mass vs removal
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.scatterplot(data=df, x='m', y='removal_pct', alpha=0.6)
plt.title('Adsorbent Mass vs Removal %')
plt.xlabel('Mass (g)'); plt.ylabel('Removal %')

# Histogram: removal distribution
plt.subplot(1,2,2)
sns.histplot(df['removal_pct'], bins=30, kde=True)
plt.title('Removal % Distribution')
plt.xlabel('Removal %')
plt.tight_layout()
plt.show()

# Correlation heatmap
plt.figure(figsize=(6,5))
corr = df[['C0','m','k_ads','q_final','C_final','removal_pct']].corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation heatmap')
plt.show()

In [ ]:
# Fit Langmuir & Freundlich to equilibrium q_final vs C_final
# Prepare data
fit_df = df_lang[['C_final','q_final']].dropna().copy()
xdata = fit_df['C_final'].values
ydata = fit_df['q_final'].values

# Langmuir fit: q = qmax * b * C / (1 + b * C) -> params: qmax, b
def langmuir_func(C, qmax, b):
    return (qmax * b * C) / (1.0 + b * C)

# Freundlich fit: q = Kf * C^(1/n) -> params: Kf, n. We'll fit as q = Kf * C**(1/n)
def freundlich_func(C, Kf, n):
    return Kf * (C ** (1.0 / n))

# Provide initial guesses to aid convergence
p0_lang = [150.0, 0.05]
p0_fre = [20.0, 2.0]

# Fit with bounds to keep parameters physical
try:
    popt_lang, pcov_lang = curve_fit(langmuir_func, xdata, ydata, p0=p0_lang, bounds=([0, 1e-6],[1000, 10]))
    popt_fre, pcov_fre = curve_fit(freundlich_func, xdata, ydata, p0=p0_fre, bounds=([0, 1.0],[1000, 10]))
    print('Langmuir fit params: qmax={:.2f}, b={:.4f}'.format(*popt_lang))
    print('Freundlich fit params: Kf={:.2f}, n={:.2f}'.format(*popt_fre))
except Exception as e:
    print('Fitting failed:', e)
    popt_lang, popt_fre = None, None

# Plot fit against data
plt.figure(figsize=(8,5))
plt.scatter(xdata, ydata, alpha=0.5, label='Synthetic equilibrium data')
C_range = np.linspace(min(xdata), max(xdata), 200)
if popt_lang is not None:
    plt.plot(C_range, langmuir_func(C_range, *popt_lang), label='Langmuir fit', linewidth=2)
if popt_fre is not None:
    plt.plot(C_range, freundlich_func(C_range, *popt_fre), label='Freundlich fit', linewidth=2)
plt.xlabel('Equilibrium Concentration C (mg/L)')
plt.ylabel('Equilibrium adsorbed amount q (mg/g)')
plt.legend()
plt.title('Isotherm fits to synthetic equilibrium data')
plt.show()

## Conclusions & Next steps

- This notebook implements kinetic adsorption simulations and generates synthetic datasets for Langmuir and Freundlich isotherms.
- You can use these synthetic datasets to train ML models, test regression routines, or design lab experiments.

**Next steps / extensions:**

1. Fit isotherm models to *real experimental data* from the Kern Canal project and compare parameters.
2. Extend the model for multi-component adsorption or continuous-flow reactors.
3. Add uncertainty quantification and measurement noise to the synthetic datasets.
4. Create an interactive dashboard (e.g., with ipywidgets or Voila) to explore parameter space in real time.

---

**How to use this notebook in your repo**: Upload it as `Simulation_notebook.ipynb`. If you keep `isotherm_functions.py` in the repo, you can replace the duplicated function cells with `from isotherm_functions import ...` to avoid code duplication.
